# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

In [2]:
cur.execute("""SELECT name FROM sqlite_master WHERE type='table';""").fetchall()

[('productlines',),
 ('offices',),
 ('customers',),
 ('employees',),
 ('orderdetails',),
 ('orders',),
 ('payments',),
 ('products',)]

## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [5]:
# Your code here
cur.execute("""SELECT lastName, firstName 
               FROM employees 
               JOIN offices 
               USING(officecode)
               WHERE city='Boston';""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,lastName,firstName
0,Firrelli,Julie
1,Patterson,Steve


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [20]:
# Your code here
cur.execute("""SELECT o.officeCode, o.city, COUNT(e.employeeNumber) AS num_employees
               FROM offices AS o
               LEFT JOIN employees AS e
               USING(officeCode)
               GROUP BY officeCode
               HAVING num_employees = 0;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,officeCode,city,num_employees
0,27,Boston,0


## Write 3 Questions of your own and answer them

In [ ]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold

In [29]:
# Display the textDescription and employee's first and last name for each product that each employee has sold
cur.execute("""SELECT textDescription, lastName, firstName 
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders o
               USING(customerNumber)
               JOIN orderdetails od
               USING(orderNumber)
               JOIN products p
               USING(productCode)
               JOIN productlines
               USING(productLine);
               """)
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,textDescription,lastName,firstName
0,The Truck and Bus models are realistic replica...,Jennings,Leslie
1,The Truck and Bus models are realistic replica...,Jennings,Leslie
2,Our Vintage Car models realistically portray a...,Jennings,Leslie
3,The Truck and Bus models are realistic replica...,Jennings,Leslie
4,Attention car enthusiasts: Make your wildest c...,Jennings,Leslie
...,...,...,...
2991,The Truck and Bus models are realistic replica...,Gerard,Martin
2992,Model trains are a rewarding hobby for enthusi...,Gerard,Martin
2993,The Truck and Bus models are realistic replica...,Gerard,Martin
2994,Our Vintage Car models realistically portray a...,Gerard,Martin


In [55]:
cur.execute("""SELECT SUM(amount) AS total_spent, customerNumber FROM payments GROUP BY customerNumber""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,total_spent,customerNumber
0,22314.36,103
1,80180.98,112
2,180585.07,114
3,116949.68,119
4,104224.79,121
...,...,...
93,77726.59,486
94,42570.37,487
95,29586.15,489
96,65541.74,495


In [63]:
# Display contactLastName, customerNumber how much each customer has spent in total
cur.execute("""SELECT contactLastName, contactFirstName, c.customerNumber, SUM(pay.amount) AS total_spent
               FROM customers c
               JOIN payments pay
               USING(customerNumber)
               GROUP BY customerNumber;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,contactLastName,contactFirstName,customerNumber,total_spent
0,Schmitt,Carine,103,22314.36
1,King,Jean,112,80180.98
2,Ferguson,Peter,114,180585.07
3,Labrune,Janine,119,116949.68
4,Bergulfsen,Jonas,121,104224.79
...,...,...,...,...
93,Salazar,Rosa,486,77726.59
94,Taylor,Sue,487,42570.37
95,Smith,Thomas,489,29586.15
96,Franco,Valarie,495,65541.74


In [65]:
cur.execute("""SELECT orderNumber, SUM(quantityOrdered) 
               FROM orderdetails 
               GROUP BY orderNumber;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,orderNumber,SUM(quantityOrdered)
0,10100,151
1,10101,142
2,10102,80
3,10103,541
4,10104,443
...,...,...
321,10421,75
322,10422,76
323,10423,111
324,10424,269


In [73]:
cur.execute("""SELECT * FROM orders
               JOIN orderdetails""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,10100,S18_1749,30,136.00,3
1,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,10100,S18_2248,50,55.09,2
2,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,10100,S18_4409,22,75.46,4
3,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,10100,S24_3969,49,35.29,1
4,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,10101,S18_2325,25,108.06,4
...,...,...,...,...,...,...,...,...,...,...,...,...
976691,10425,2005-05-31,2005-06-07,,In Process,,119,10425,S24_2300,49,127.79,9
976692,10425,2005-05-31,2005-06-07,,In Process,,119,10425,S24_2840,31,31.82,5
976693,10425,2005-05-31,2005-06-07,,In Process,,119,10425,S32_1268,41,83.79,11
976694,10425,2005-05-31,2005-06-07,,In Process,,119,10425,S32_2509,11,50.32,6


In [71]:
#Display the total amount of orders each customer made
cur.execute("""SELECT contactLastName, contactFirstName, c.customerNumber, COUNT(orderNumber) AS total_orders
               FROM customers c
               JOIN orders o
               USING(customerNumber)
               GROUP BY o.customerNumber;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,contactLastName,contactFirstName,customerNumber,total_orders
0,Schmitt,Carine,103,3
1,King,Jean,112,3
2,Ferguson,Peter,114,5
3,Labrune,Janine,119,4
4,Bergulfsen,Jonas,121,4
...,...,...,...,...
93,Salazar,Rosa,486,3
94,Taylor,Sue,487,2
95,Smith,Thomas,489,2
96,Franco,Valarie,495,2


In [72]:
# Display how total products each customers has bought 
# cur.execute("""SELECT contactLastName, contactFirstName, c.customerNumber, SUM(quantityOrdered) AS total_products
#                FROM customers c
#                WHERE JOIN orders o
#                USING(customerNumber)
#                GROUP BY o.customerNumber
#                JOIN orderdetails od
#                USING(orderNumber)
#                GROUP BY orderNumber;""")
# df = pd.DataFrame(cur.fetchall())
# df.columns = [i[0] for i in cur.description]
# df

OperationalError: near "JOIN": syntax error

## Level Up: Display the names of every individual product that each employee has sold

In [ ]:
# Your code here

## Level Up: Display the Number of Products each employee has sold

In [ ]:
# Your code here

## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!